In [1]:
debugging = True
IPTS = 17685

# Description 

Load a stack of images, select a region and then the average counts of the region vs the stack will be displayed

# Imports 

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
!pyuic4 ui/ui_display_counts_of_region_vs_stack.ui -o __code/ui_display_counts_of_region_vs_stack.py

In [4]:
%gui qt

In [5]:
try:
    from PyQt4.QtGui import QFileDialog
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QMainWindow
except ImportError:
    from PyQt5.QtWidgets import QFileDialog
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QApplication

In [6]:
import numbers

from __code import file_handler, utilities, fileselector
from NeuNorm.normalization import Normalization
from neutronbraggedge.experiment_handler import *

if debugging:
    ipts = IPTS
else:
    ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts, debugging=debugging)
print("Working dir: {}".format(working_dir))

Working dir: /Volumes/my_book_thunderbolt_duo/IPTS/IPTS_17685


# Select Stack Folder

In [7]:
input_folder_ui = fileselector.FileSelectorPanel(instruction='Select Input Folder', type='directory', start_dir=working_dir, multiple=False)
input_folder_ui.show()

# Load Stack

In [8]:
working_folder = input_folder_ui.result_full_path()
o_norm = Normalization()
o_norm.load(folder=working_folder, notebook=True)

# Select Region and display Counts vs file_index (or time)

In [11]:
import pyqtgraph as pg
from pyqtgraph.dockarea import *

from __code.ui_display_counts_of_region_vs_stack import Ui_MainWindow as UiMainWindow


class ImageWindow(QMainWindow):

    stack = []
    integrated_stack = []
    working_folder = ''
    x_axis = {'label': 'File Index', 'type': 'file_index', 'data': []}
    y_axis = {'label': 'Mean Counts', 'data': []}
    spectra_file = ''
    
    def __init__(self, parent=None, stack=[], working_folder=''):
        QMainWindow.__init__(self, parent=parent)
        self.ui = UiMainWindow()
        self.ui.setupUi(self)
        self.setWindowTitle("Select Rotation Angle for All Images")

        self.stack = np.array(stack)
        self.integrated_stack = self.stack.sum(axis=0)
        self.working_folder = working_folder

        self.initialize_pyqtgraph()
        self.init_label()

        self.display_image()
        self.update_x_axis()
        self.roi_changed()
        
    def init_label(self):
        _tof_label = u"TOF (\u00B5s)"
        self.ui.tof_radio_button.setText(_tof_label)
        _lambda_label = u"lambda (\u212B)"
        self.ui.lambda_radio_button.setText(_lambda_label)
        _offset_label = u"\u00B5s"
        self.ui.detector_offset_units.setText(_offset_label)

    def display_image(self):
        self.ui.image_view.setImage(self.integrated_stack)

    def plot(self):
        x_axis_data = self.x_axis['data']
        x_axis_label = self.x_axis['label']
        
        y_axis_data = self.y_axis['data']
        y_axis_label = self.y_axis['label']
        
        print(len(y_axis_data))
        
        x_axis_data = x_axis_data[0: len(y_axis_data)]
        
        self.counts_vs_index.clear()
        self.counts_vs_index.plot(x_axis_data, y_axis_data)
        
        self.counts_vs_index.setLabel('bottom', x_axis_label)
        self.counts_vs_index.setLabel('left', y_axis_label)
        
    def initialize_pyqtgraph(self):
        area = DockArea()
        area.setVisible(True)
        d1 = Dock("Image Integrated Preview", size=(200, 300))
        d2 = Dock("Counts vs Image Index of Selection", size=(200, 100))

        area.addDock(d1, 'top')
        area.addDock(d2, 'bottom')

        preview_widget = pg.GraphicsLayoutWidget()
        pg.setConfigOptions(antialias=True)

        # image view
        self.ui.image_view = pg.ImageView()
        self.ui.image_view.ui.menuBtn.hide()
        self.ui.image_view.ui.roiBtn.hide()

        # default ROI
        self.ui.roi = pg.ROI(
            [0, 0], [20, 20], pen=(62, 13, 244), scaleSnap=True)  #blue
        self.ui.roi.addScaleHandle([1, 1], [0, 0])
        self.ui.image_view.addItem(self.ui.roi)
        self.ui.roi.sigRegionChanged.connect(self.roi_changed)
        d1.addWidget(self.ui.image_view)

        self.counts_vs_index = pg.PlotWidget(title='')
        self.counts_vs_index.plot()
        d2.addWidget(self.counts_vs_index)

        vertical_layout = QtGui.QVBoxLayout()
        vertical_layout.addWidget(area)

        self.ui.widget.setLayout(vertical_layout)

    def roi_changed(self):
        region = self.ui.roi.getArraySlice(self.integrated_stack,
                                           self.ui.image_view.imageItem)
        x0 = region[0][0].start
        x1 = region[0][0].stop - 1
        y0 = region[0][1].start
        y1 = region[0][1].stop - 1

        mean_selection = [_data[y0:y1, x0:x1].mean() for _data in self.stack]
        self.y_axis['data'] = mean_selection
        self.plot()

    # x_axis
    def get_x_axis_selected(self):
        if self.ui.file_index_ratio_button.isChecked():
            return 'file_index'
        elif self.ui.tof_radio_button.isChecked():
            return 'tof'
        else:
            return 'lambda'

    def update_x_axis(self):
        x_axis_selected = self.get_x_axis_selected()

        spectra_file = self.spectra_file
        if not os.path.exists(spectra_file):
            x_axis_selected = 'file_index'

        distance_source_detector = self.ui.distance_source_detector_value.text()
        if not isinstance(distance_source_detector, numbers.Number):
            x_axis_selected = 'file_index'

        detector_offset = self.ui.detector_offset_value.text()
        if not isinstance(detector_offset, numbers.Number):
            x_axis_selected = 'file_index'

        self.x_axis['type'] = x_axis_selected
        if x_axis_selected == 'file_index':
            self.x_axis['data'] = np.arange(len(self.integrated_stack))
            self.x_axis['label'] = 'File Index'
        else:
            _tof_handler = TOF(filename=spectra_file)
            if x_axis_selected == 'tof':
                self.x_axis['data'] = _tof_handler.tof_array
                self.x_axis['label'] = u'TOF (/u00B5s)'
            else:
                _exp = Experiment(tof = _tof_handler.tof_array, 
                                  display_tance_source_detector_m = distance_source_detector,
                                  detector_offset_micros = detector_offset)
                self.x_axis['data'] = _exp.lambda_array
                self.x_axis['label'] = u'\u0394\u03BB:'
                
    def distance_source_detector_validated(self):
        pass

    def detector_offset_validated(self):
        pass

    def time_spectra_file_browse_button_clicked(self):
        spectra_file = QFileDialog.getOpenFileName(
            caption='Select Time Spectra',
            directory=self.working_folder,
            filter='txt (*_Spectra.txt);;All (*.*)')
        if spectra_file:
            self.ui.time_spectra_file.setText(os.path.basename(spectra_file))
            self.spectra_file = spectra_file
            self.update_x_axis()
            self.plot()

    def done_button_clicked(self):
        self.close()

    def closeEvent(self, event=None):
        pass


_image = ImageWindow(
    stack=(o_norm.data['sample']['data']), working_folder=working_folder)
_image.show()

186


In [ ]:
from __code.ui_display_counts_of_region_vs_stack import Ui_MainWindow as UiMainWindow

class ImageWindow(QMainWindow):
    
    grid_size = 100
    live_data = []
    
    def __init__(self, parent=None):
        QMainWindow.__init__(self, parent=parent)
        self.ui = UiMainWindow()
        self.ui.setupUi(self)
        self.init_statusbar()
        self.setWindowTitle("Select Rotation Angle for All Images")
        
        self.ui.image_view = pg.ImageView()
        self.ui.image_view.ui.roiBtn.hide()
        self.ui.image_view.ui.menuBtn.hide()
        
        bottom_layout = QtGui.QHBoxLayout()

        # file index slider
        label_1 = QtGui.QLabel("File Index")
        self.ui.slider = QtGui.QSlider(QtCore.Qt.Horizontal)
        self.ui.slider.setMaximum(len(list_images)-1)
        self.ui.slider.setMinimum(0)
        self.ui.slider.valueChanged.connect(self.file_index_changed)
        
        # spacer
        spacer = QtGui.QSpacerItem(40, 20, QtGui.QSizePolicy.Expanding, QtGui.QSizePolicy.Minimum)

        # rotation value
        label_2 = QtGui.QLabel("Rotation (degrees)")
        self.ui.rotation_value = QtGui.QLineEdit('0')
        self.ui.rotation_value.setMaximumWidth(50)
        self.ui.rotation_value.returnPressed.connect(self.rotation_value_changed)
        
        bottom_layout.addWidget(label_1)
        bottom_layout.addWidget(self.ui.slider)
        bottom_layout.addItem(spacer)
        bottom_layout.addWidget(label_2)
        bottom_layout.addWidget(self.ui.rotation_value)

        bottom_widget = QtGui.QWidget()
        bottom_widget.setLayout(bottom_layout)
        
        vertical_layout = QtGui.QVBoxLayout()
        vertical_layout.addWidget(self.ui.image_view)
        vertical_layout.addWidget(bottom_widget)

        self.ui.widget.setLayout(vertical_layout)
        
 #       self.rotated_working_data = working_data
 #       self.working_data = working_data
        
        self.init_imageview()
#        self.display_grid()
#        self.display_crop_region()
        
    def init_statusbar(self):
        self.eventProgress = QtGui.QProgressBar(self.ui.statusbar)
        self.eventProgress.setMinimumSize(20, 14)
        self.eventProgress.setMaximumSize(540, 100)
        self.eventProgress.setVisible(False)
        self.ui.statusbar.addPermanentWidget(self.eventProgress)
        
    def display_crop_region(self):
        [image_height, image_width] = self.get_image_dimension(self.rotated_working_data)
        x0, y0 = 0, 0
        width = image_width-1
        height = image_height-1
        
        roi = pg.ROI([x0, y0], [height, width], pen=(209, 230, 27), scaleSnap=True)
        roi.addScaleHandle([1,1], [0,0])
        self.ui.image_view.addItem(roi)
        self.roi = roi
        
    def get_image_dimension(self, array_image):
        if len(np.shape(array_image)) > 2:
            return np.shape(array_image[0])
        else:
            return np.shape(array_image)
        
    def get_selected_image(self, file_index):

        if len(np.shape(working_data)) > 2:
            return self.working_data[file_index]
        else:
            return self.working_data
        
    def display_grid(self):
        [width, height] = self.get_image_dimension(self.rotated_working_data)
        bin_size = self.grid_size
        x0 = 0
        y0 = 0
        
        pos_adj_dict = {}

        nbr_height_bins = np.float(height) / np.float(bin_size)
        real_height = y0 + np.int(nbr_height_bins) * np.int(bin_size)
        
        nbr_width_bins = np.float(width) / np.float(bin_size)
        read_width = x0 + np.int(nbr_width_bins) * np.int(bin_size)
        
        # pos (each matrix is one side of the lines)
        pos = []
        adj = []

        # vertical lines
        x = x0
        index = 0
        while (x <= x0 + width):
            one_edge = [x, y0]
            other_edge = [x, real_height]
            pos.append(one_edge)
            pos.append(other_edge)
            adj.append([index, index+1])
            x += bin_size
            index += 2
            
        # horizontal lines
        y = y0
        while (y <= y0 + height):
            one_edge = [x0, y]
            other_edge = [read_width, y]
            pos.append(one_edge)
            pos.append(other_edge)
            adj.append([index, index+1])
            y += bin_size
            index += 2

        pos = np.array(pos)
        adj = np.array(adj)
        
        line_color = (255, 0, 0, 155, 0.2)
        lines = np.array([line_color for n in np.arange(len(pos))],
                             dtype=[('red',np.ubyte),('green',np.ubyte),
                                    ('blue',np.ubyte),('alpha',np.ubyte),
                                   ('width',float)]) 
    
        line_view_binning = pg.GraphItem()
        self.ui.image_view.addItem(line_view_binning)
        line_view_binning.setData(pos = pos,
                                 adj = adj,
                                 pen = lines,
                                 symbol = None,
                                 pxMode = False)
        
        self.line_view_binning = line_view_binning

    def init_imageview(self):
        image = self.get_selected_image(0)
        self.ui.image_view.setImage(image)
        self.live_data = image
        
    def rotation_value_changed(self):
        _rotation_value = np.float(str(self.ui.rotation_value.text()))
        _file_index = self.ui.slider.value()
    
        rotated_data = scipy.ndimage.interpolation.rotate(self.get_selected_image(_file_index), _rotation_value)
        self.live_data = rotated_data
        self.ui.image_view.removeItem(self.line_view_binning)
        self.display_grid()
        self.ui.image_view.setImage(rotated_data)
    
    def rotate_and_crop_all(self):
        global rotated_working_data            

        #FIXME
        #inform user that the window will close by itself and it's still doing something
        
        region = self.roi.getArraySlice(self.live_data,
                                        self.ui.image_view.imageItem)
                                        
        x0 = region[0][0].start
        x1 = region[0][0].stop-1
        y0 = region[0][1].start
        y1 = region[0][1].stop-1
        
        self.eventProgress.setValue(0)
        self.eventProgress.setMaximum(len(self.working_data))
        self.eventProgress.setVisible(True)
        
        _rotation_value = np.float(str(self.ui.rotation_value.text()))
        self.rotated_working_data = []
        
        if len(np.shape(self.working_data)) == 2:
            self.working_data = [self.working_data]
        
        for _index, _data in enumerate(self.working_data):
            # rotate image
            rotated_data = scipy.ndimage.interpolation.rotate(_data, _rotation_value)
            rotated_data = rotated_data[y0:y1+1, x0:x1+1]
            self.rotated_working_data.append(rotated_data)
            self.eventProgress.setValue(_index+1)
            QtGui.QApplication.processEvents()
    
        self.file_index_changed()
        self.ui.image_view.removeItem(self.line_view_binning)
        self.display_grid()
        
        rotated_working_data = self.rotated_working_data
        self.eventProgress.setVisible(False)
    
    def apply_clicked(self):
        self.rotate_and_crop_all()
        self.close()
        
    def cancel_clicked(self):
        self.close()
    
    def file_index_changed(self):
        self.rotation_value_changed()
        
    def display_image(self, image):
        self.ui.image_view.setImage(image)
        
    def closeEvent(self, event=None):
        global rotation_angle
        rotation_angle = np.float(str(self.ui.rotation_value.text()))
        
_image = ImageWindow()
_image.show()


display(HTML('<span style="font-size: 20px; color:blue">Select the rotation angle in the UI that poped up \
(maybe hidden behind this browser!)</span>'))

# Export Images 

In [8]:
display(HTML('<span style="font-size: 20px; color:blue">Select the folder that will contain the rotated images!</span>'))

output_folder = gui_widgets.gui_dname(dir=working_dir, message='Select the Output Folder ...')

if output_folder:
        
    w = widgets.IntProgress()
    w.max = len(list_images)
    display(w)
    
    for _index, _file in enumerate(list_images):
        _base_name = os.path.basename(_file)
        [base, ext] = os.path.splitext(_base_name)
        _full_file_name = os.path.join(output_folder, base + '_rot_{}'.format(np.int(rotation_angle)) + '.tiff')
        file_handler.make_tiff(data=rotated_working_data[_index], filename=_full_file_name)
    
        w.value = _index + 1